In [56]:
import pandas as pd
import json

df = pd.read_csv("./estate_data/naver_region_name_estate.csv")
with open("./estate_data/estate_column_info.json", "r", encoding="utf-8") as f:
    column_info = json.load(f)
print(column_info)

print(df.shape)
print('----------------------')
# print(df.columns)
print('----------------------')
# print(df.isnull().sum())
print('----------------------')


{'atclNo': '매물 번호', 'atclNm': '건물명', 'region': '동 이름', 'cortarNo': '지역 코드', 'rletTpNm': '매물 종류', 'tradTpNm': '거래 종류', 'flrInfo': '층수 정보', 'prc': '보증금', 'rentPrc': '월세', 'spc1': '공급면적', 'spc2': '전용면적', 'direction': '방향', 'atclCfmYmd': '확인 일자', 'lat': '위도', 'lng': '경도', 'atclFetrDesc': '특징 설명', 'bildNm': '동', 'rltrNm': '부동산 이름'}
(2426, 17)
----------------------
----------------------
----------------------


In [ ]:


'''
print(df['region'].unique())
print(df['region'].nunique())
print(df['region'].value_counts())
'''

# region만으로 그룹화 (전체 평균)
df_region_mean = df.groupby('region')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_region_mean['prc/spc1'] = df_region_mean['prc'] / df_region_mean['spc1']
df_region_mean['rentPrc/spc1'] = df_region_mean['rentPrc'] / df_region_mean['spc1']
df_region_mean['prc/spc2'] = df_region_mean['prc'] / df_region_mean['spc2']
df_region_mean['rentPrc/spc2'] = df_region_mean['rentPrc'] / df_region_mean['spc2']

df_region_mean.columns = pd.MultiIndex.from_product([['Tot_mean'], df_region_mean.columns])
df_grouped = df.groupby(['region', 'rletTpNm'])[['prc', 'rentPrc', 'spc1', 'spc2']].mean()
df_pivot = df_grouped.unstack(level='rletTpNm')

# 통합: 컬럼 합치기
df_final = pd.concat([df_region_mean, df_pivot], axis=1)
print(df_final.head())

df_final.to_csv("./estate_data/df_price_overall.csv", encoding="utf-8-sig")


                      Tot_mean                                                \
                           prc     rentPrc       spc1       spc2    prc/spc1   
region                                                                         
금호1가동             15439.473684  122.236842  73.918919  57.805263  208.870394   
금호2가동             18457.894737  170.000000  84.833333  62.469474  217.578327   
금호3가동             18823.529412  206.843137  93.686275  70.816275  200.920887   
금호4가동              7600.000000  110.955556  62.750000  50.119556  121.115538   
기타지역(1120012200)   9110.215054   95.946237  55.174863  38.040484  165.115317   

                                                                 prc  \
                 rentPrc/spc1    prc/spc2 rentPrc/spc2           다세대   
region                                                                 
금호1가동                1.653661  267.094601     2.114632           NaN   
금호2가동                2.003929  295.470630     2.721329           NaN   

In [90]:
# --------- 지역별 -> 보증금(prc), 월세(rentPrc), 공급면적(spc1), 전용면적(spc2),
#  면적별 보증금, 면적별 월세 지정 후 csv로 저장  -------------------------------


# df_region : 각 지역별로 나눈 데이터프레임. ex) '성수1가제2동' : ~~ / '옥수동' : ~~ 
df_region = {val: df[df['region'] == val] for val in df['region'].unique()}

# 숫자로 변환 (쉼표, 문자열 등 제거)
df['spc1'] = pd.to_numeric(df['spc1'], errors='coerce')
df['spc2'] = pd.to_numeric(df['spc2'], errors='coerce')

# 이제 그룹화
df_price = df.groupby('region')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_price['prc/spc1'] = df_price['prc'] / df_price['spc1']
df_price['rentPrc/spc1'] = df_price['rentPrc'] / df_price['spc1']
df_price['prc/spc2'] = df_price['prc'] / df_price['spc2']
df_price['rentPrc/spc2'] = df_price['rentPrc'] / df_price['spc2']

df_price.to_csv("./estate_data/df_region_price.csv", encoding="utf-8-sig")

In [ ]:
# --------- 매물 종류 (아파트, 빌라 etc) 별 -> 보증금(prc), 월세(rentPrc), 공급면적(spc1),
#  전용면적(spc2), 면적별 보증금, 면적별 월세 지정 후 csv로 저장 -------------------------

df_BdTp= df.groupby('rletTpNm')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_BdTp['prc/spc1'] = df_BdTp['prc'] / df_BdTp['spc1']
df_BdTp['rentPrc/spc1'] = df_BdTp['rentPrc'] / df_BdTp['spc1']
df_BdTp['prc/spc2'] = df_BdTp['prc'] / df_BdTp['spc2']
df_BdTp['rentPrc/spc2'] = df_BdTp['rentPrc'] / df_BdTp['spc2']

df_BdTp.to_csv("./estate_data/df_type_price.csv", encoding="utf-8-sig")

In [ ]:
# --------- 지역 + 매물 종류 (아파트, 빌라 etc) 별 -> 보증금(prc), 월세(rentPrc), 공급면적(spc1),
#  전용면적(spc2), 면적별 보증금, 면적별 월세 지정 후 csv로 저장 ----------------------

# region만으로 그룹화 (전체 평균)
df_region_mean = df.groupby('region')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_region_mean['prc/spc1'] = df_region_mean['prc'] / df_region_mean['spc1']
df_region_mean['rentPrc/spc1'] = df_region_mean['rentPrc'] / df_region_mean['spc1']
df_region_mean['prc/spc2'] = df_region_mean['prc'] / df_region_mean['spc2']
df_region_mean['rentPrc/spc2'] = df_region_mean['rentPrc'] / df_region_mean['spc2']

df_region_mean.columns = pd.MultiIndex.from_product([['Tot_mean'], df_region_mean.columns])
df_grouped = df.groupby(['region', 'rletTpNm'])[['prc', 'rentPrc', 'spc1', 'spc2']].mean()
df_pivot = df_grouped.unstack(level='rletTpNm')

# 통합: 컬럼 합치기
print(df_pivot.head())

df_pivot.to_csv("./estate_data/df_region_type_price.csv", encoding="utf-8-sig")

                           prc                                          \
rletTpNm                   다세대       단독/다가구            빌라         상가주택   
region                                                                   
금호1가동                      NaN   400.000000  12208.333333          NaN   
금호2가동                      NaN  3500.000000   8200.000000  2000.000000   
금호3가동                      NaN  6000.000000   5000.000000  6111.111111   
금호4가동             11666.666667  2447.368421   4375.000000   500.000000   
기타지역(1120012200)           NaN  4011.627907   3500.000000  4000.000000   

                                                                           \
rletTpNm                   아파트   아파트분양권  연립         오피스텔           원룸 재건축   
region                                                                      
금호1가동             19086.956522      NaN NaN          NaN          NaN NaN   
금호2가동             25921.052632      NaN NaN  1800.000000   650.000000 NaN   
금호3가동             2430

In [95]:
# ----------- 지역 별 주거 형태 숫자 평균 ------------

# region만으로 그룹화 (전체 평균)
df_region_rletTpNm_num = df.groupby(['region', 'rletTpNm']).size().unstack(fill_value=0)

# 통합: 컬럼 합치기
print(df_region_rletTpNm_num.head())

df_region_rletTpNm_num.to_csv("./estate_data/df_region_type_count.csv", encoding="utf-8-sig")

rletTpNm          다세대  단독/다가구  빌라  상가주택  아파트  아파트분양권  연립  오피스텔  원룸  재건축
region                                                                 
금호1가동               0       3  12     0   23       0   0     0   0    0
금호2가동               0       3   5     2   38       0   0     5   4    0
금호3가동               0       3   1     9   36       0   0     0   2    0
금호4가동               3      19   4     1   15       0   0     0   3    0
기타지역(1120012200)    0      43  28     2    0      35   0    68  10    0


In [ ]:
# 공급면적(spc) 및 전용면적(spc) 별 가격 csv로 저장 (v)
# 지역에 따른 주거 형태(아파트, 주택 등) 별 가격 csv로 저장 (v)
# 지역 별 주거 형태에 따른 가격 평균 (v)
# 지역 별 주거 형태 숫자 평균 (v)
# 지역 별 면적 평균 (v)


# --------------------------- 통합 코드 ---------------------------------
import pandas as pd
import json

df = pd.read_csv("./estate_data/naver_region_name_estate.csv")
with open("./estate_data/estate_column_info.json", "r", encoding="utf-8") as f:
    column_info = json.load(f)

df_region_mean = df.groupby('region')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_region_mean['prc/spc1'] = df_region_mean['prc'] / df_region_mean['spc1']
df_region_mean['rentPrc/spc1'] = df_region_mean['rentPrc'] / df_region_mean['spc1']
df_region_mean['prc/spc2'] = df_region_mean['prc'] / df_region_mean['spc2']
df_region_mean['rentPrc/spc2'] = df_region_mean['rentPrc'] / df_region_mean['spc2']

df_region_mean.columns = pd.MultiIndex.from_product([['Tot_mean'], df_region_mean.columns])
df_grouped = df.groupby(['region', 'rletTpNm'])[['prc', 'rentPrc', 'spc1', 'spc2']].mean()
df_pivot = df_grouped.unstack(level='rletTpNm')

# 통합: 컬럼 합치기
df_final = pd.concat([df_region_mean, df_pivot], axis=1)

df_final.to_csv("./estate_data/df_price_overall.csv", encoding="utf-8-sig")

# --------- 매물 종류 (아파트, 빌라 etc) 별 -> 보증금(prc), 월세(rentPrc), 공급면적(spc1),
#  전용면적(spc2), 면적별 보증금, 면적별 월세 지정 후 csv로 저장 -------------------------

df_BdTp= df.groupby('rletTpNm')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_BdTp['prc/spc1'] = df_BdTp['prc'] / df_BdTp['spc1']
df_BdTp['rentPrc/spc1'] = df_BdTp['rentPrc'] / df_BdTp['spc1']
df_BdTp['prc/spc2'] = df_BdTp['prc'] / df_BdTp['spc2']
df_BdTp['rentPrc/spc2'] = df_BdTp['rentPrc'] / df_BdTp['spc2']

df_BdTp.to_csv("./estate_data/df_type_price.csv", encoding="utf-8-sig")

# --------- 지역별 -> 보증금(prc), 월세(rentPrc), 공급면적(spc1), 전용면적(spc2),
#  면적별 보증금, 면적별 월세 지정 후 csv로 저장  -------------------------------


# df_region : 각 지역별로 나눈 데이터프레임. ex) '성수1가제2동' : ~~ / '옥수동' : ~~ 
df_region = {val: df[df['region'] == val] for val in df['region'].unique()}

# 숫자로 변환 (쉼표, 문자열 등 제거)
df['spc1'] = pd.to_numeric(df['spc1'], errors='coerce')
df['spc2'] = pd.to_numeric(df['spc2'], errors='coerce')

# 이제 그룹화
df_price = df.groupby('region')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_price['prc/spc1'] = df_price['prc'] / df_price['spc1']
df_price['rentPrc/spc1'] = df_price['rentPrc'] / df_price['spc1']
df_price['prc/spc2'] = df_price['prc'] / df_price['spc2']
df_price['rentPrc/spc2'] = df_price['rentPrc'] / df_price['spc2']

df_price.to_csv("./estate_data/df_region_price.csv", encoding="utf-8-sig")

# --------- 지역 + 매물 종류 (아파트, 빌라 etc) 별 -> 보증금(prc), 월세(rentPrc), 공급면적(spc1),
#  전용면적(spc2), 면적별 보증금, 면적별 월세 지정 후 csv로 저장 ----------------------

# region만으로 그룹화 (전체 평균)
df_region_mean = df.groupby('region')[['prc', 'rentPrc', 'spc1', 'spc2']].mean()

df_region_mean['prc/spc1'] = df_region_mean['prc'] / df_region_mean['spc1']
df_region_mean['rentPrc/spc1'] = df_region_mean['rentPrc'] / df_region_mean['spc1']
df_region_mean['prc/spc2'] = df_region_mean['prc'] / df_region_mean['spc2']
df_region_mean['rentPrc/spc2'] = df_region_mean['rentPrc'] / df_region_mean['spc2']

df_region_mean.columns = pd.MultiIndex.from_product([['Tot_mean'], df_region_mean.columns])
df_grouped = df.groupby(['region', 'rletTpNm'])[['prc', 'rentPrc', 'spc1', 'spc2']].mean()
df_pivot = df_grouped.unstack(level='rletTpNm')

df_pivot.to_csv("./estate_data/df_region_type_price.csv", encoding="utf-8-sig")

# ----------- 지역 별 주거 형태 숫자 평균 ------------

# region만으로 그룹화 (전체 평균)
df_region_rletTpNm_num = df.groupby(['region', 'rletTpNm']).size().unstack(fill_value=0)

df_region_rletTpNm_num.to_csv("./estate_data/df_region_type_count.csv", encoding="utf-8-sig")